In [35]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc
import pandas as pd
import plotly.express as px

# =====================
# CARGA Y PROCESAMIENTO DE DATOS
# =====================

# Ruta del archivo
file_path = "C:\\Users\\Ruth Rolón Aranda\\Documents\\Cambra\\Analisis para la consultoria\\ejemplo dash\\finanzas\\guaranies.xlsx"
df = pd.read_excel(file_path)

# Mapeo de calificaciones a valores numéricos (más seguros = menor número)
riesgo_map = {
    'AAA+py': 1, 'AAApy': 2, 'AAA-py': 3,
    'AA+py': 4, 'AApy': 5, 'AA-py': 6,
    'A+py': 7, 'Apy': 8, 'A-py': 9,
    'BBB+py': 10, 'BBBpy': 11, 'BBB-py': 12,
    'BB+py': 13, 'BBpy': 14, 'BB-py': 15,
    'B+py': 16, 'Bpy': 17, 'B-py': 18
}
df['Riesgo_Num'] = df['Riesgo'].map(riesgo_map)

# Plazo como número y clasificación
df['Plazo'] = pd.to_numeric(df['Plazo'], errors='coerce')

def clasificar_plazo(dias):
    if dias <= 7:
        return 'Vista'
    elif dias <= 90:
        return 'Muy corto plazo'
    elif dias <= 365:
        return 'Corto plazo'
    elif dias <= 900:
        return 'Mediano plazo'
    else:
        return 'Largo plazo'

df['Plazo_Tipo'] = df['Plazo'].apply(clasificar_plazo)

# Reemplazo de "na" por 1.000.000 y valores nulos
df['Capital Minimo'] = df['Capital Minimo'].replace("na", 1_000_000)
df['Capital Minimo'] = pd.to_numeric(df['Capital Minimo'], errors='coerce').fillna(1_000_000)

# Crear campo combinado para el hover
df['Entidad+Producto'] = df['Entidad'] + " – " + df['Producto']

# =====================
# GRÁFICO
# =====================

fig = px.scatter(
    df,
    x="Riesgo_Num",
    y="Tasa MAX",
    color="Categoria",
    symbol="Plazo_Tipo",
    hover_name="Entidad+Producto",
    hover_data={
        "Riesgo_Num": False,
        "Tasa MAX": True,
        "Categoria": True,
        "Plazo_Tipo": True,
        "Riesgo": True,
        "Capital Minimo": True,
    },
    title="Relación entre riesgo, rentabilidad y tipo de producto financiero",
    labels={
        "Riesgo_Num": "Riesgo",
        "Tasa MAX": "Tasa de interés máxima (%)"
    },
    #size="Capital Minimo",
    size_max=60,
    template="plotly_white"
)

fig.update_layout(
    # Si querés mostrar a los activos más seguros a la derecha, activá esta línea:
    # xaxis=dict(autorange="reversed"),
    legend_title_text="Producto financiero",
    title_font=dict(size=20)
)

# =====================
# APP DASH
# =====================

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    # Encabezado
    html.Header(
        dbc.Container([
            dbc.Row([
                dbc.Col(
                    dbc.Row([
                        dbc.Col(html.Img(src='/assets/CAMBRA.png', style={'height': '80px'}), width="auto"),
                        dbc.Col(html.H1(
                            "Guía de Productos Financieros",
                            className="m-0",
                            style={
                                'font-family': 'Avenir, sans-serif',
                                'font-weight': '700',
                                'font-size': '2rem',
                                'text-align': 'center',
                                'color': '#333'
                            }
                        ), align="center")
                    ], align="center"),
                    width=10
                )
            ], justify="center", className="py-3")
        ]),
        style={'backgroundColor': 'white'},
    ),

    # Gráfico
    dbc.Container([
        dcc.Graph(figure=fig)
    ])
])

if __name__ == "__main__":
    app.run_server(debug=True)



---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

